Il problema del set covering (SCP) ha in input un insieme N di elementi e un insieme di
sottinsiemi di N (N ⊆ 2N). Ad ogni sottinsieme è associato un costo c_ij e si vogliono
selezionare i sottinsiemi che coprano tutti gli elementi di N a costo minimo

In [146]:
from random import random, seed
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

from tqdm.auto import tqdm
import numpy as np

In [147]:
seed(7)
PROBLEM_SIZE = 70
NUM_SETS = 100
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

SETS = set(tuple(tuple(set) for set in SETS))
SETS = list(np.array(set) for set in SETS)

SETS.sort(key=lambda x: -sum(x))
SETS = tuple(SETS)

NUM_SETS = len(SETS)

State = namedtuple('State', ['taken', 'not_taken'])

len(SETS)

100

In [148]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

In [149]:
def goal_check(state):
    return np.all(reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))


def distance(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return Num_EmptySpaces

def distance_Convex(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return 0.1*Num_EmptySpaces+0.9*Num_SetsTaken

def distance_A_star(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return Num_SetsTaken + Num_EmptySpaces/PROBLEM_SIZE

def distance_stochastic(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return (Num_SetsTaken/(PROBLEM_SIZE-Num_EmptySpaces+1))+(1-(0.5)**(Num_EmptySpaces))

def distance_stochastic_1(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    p = (2**(PROBLEM_SIZE-Num_EmptySpaces))/(2**PROBLEM_SIZE-Num_SetsTaken)
    return (Num_SetsTaken/(PROBLEM_SIZE-Num_EmptySpaces+1))+(1-p)


def distance_stochastic_2(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    p = (2**(PROBLEM_SIZE-Num_EmptySpaces))/(2**PROBLEM_SIZE-Num_SetsTaken)
    return (1-p)



def distance_deterministic(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return (Num_SetsTaken/(PROBLEM_SIZE-Num_EmptySpaces+1))+h4(state)/(PROBLEM_SIZE-Num_SetsTaken+1)




# A* search

In [150]:

def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def h4(state):
    already_covered = covered(state)
    index = list()
    if np.all(already_covered):
        return 0
    for i in range(PROBLEM_SIZE):
        if already_covered[i] == False:
            index.append(i)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum([SETS[h][k] for k in index]) for h in state.not_taken),reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


def f(state):
    return len(state.taken) + h3(state)

def f1(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return (Num_SetsTaken/(PROBLEM_SIZE-Num_EmptySpaces+1)) + (1-(0.5)**h3(state))

def f2(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return (Num_SetsTaken/(PROBLEM_SIZE-Num_EmptySpaces+1)) + h4(state)

def distance_deterministic(state):
    Num_EmptySpaces = PROBLEM_SIZE - np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    Num_SetsTaken = len(state.taken)
    return (Num_SetsTaken/(PROBLEM_SIZE-Num_EmptySpaces+1))+h4(state)/(PROBLEM_SIZE-Num_SetsTaken+1)




In [151]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Probelm not solvable"

In [152]:
frontier = PriorityQueue()
#frontier = SimpleQueue()
#frontier = LifoQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f1(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f1(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)
    pbar.close
print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

1it [00:00,  4.40it/s]

230it [00:56,  4.11it/s]

Solved in 230 steps (5 tiles)


In [153]:
current_state

State(taken={64, 1, 9, 14, 17}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99})

In [154]:
sum(sum(SETS[i] for i in current_state.taken))

130

In [155]:
frontier = PriorityQueue()
#frontier = SimpleQueue()
#frontier = LifoQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f2(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f2(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)
    pbar.close
print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

0it [00:00, ?it/s]

230it [00:35,  6.43it/s]

Solved in 230 steps (5 tiles)


In [156]:
current_state

State(taken={64, 1, 9, 14, 17}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99})

In [161]:
frontier = PriorityQueue()
#frontier = SimpleQueue()
#frontier = LifoQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((distance_stochastic(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((distance_stochastic(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)
    pbar.close
print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

388it [00:03, 116.05it/s]

Solved in 388 steps (5 tiles)


In [162]:
current_state

State(taken={64, 1, 9, 14, 17}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99})

In [159]:
x=[False, False, True, True, False]
ind = list()
for i in range(5):
        if x[i] == False:
            ind.append(i)
            
ind    

      

[0, 1, 4]

In [160]:
y=[[True, True, False, False, False],[True, True, False, False, True],[False, False, True, False, False]]
sorted((sum([y[h][k] for k in ind]) for h in range(3)),reverse=True)


[3, 2, 0]

# 
##
###